# Get data from excel using Pandas

- loop through rows and create observations for each one and validate and save

In [4]:
import pandas as pd
from pathlib import Path

source_path = '/Users/ehaas/Documents/FHIR/Davinci-CDEX/my-notes/attachment example data.xlsx'
# file_name = 'prov_data.csv'#'a1c_data.csv' #'wt_data.csv'
path = Path.cwd() / source_path

my_row = 3

#df = pd.read_('wt_data.csv', skip_blank_lines=True)
df = pd.read_excel(path,)
df = df.astype(str)
#df.dropna(how="all", inplace=True)
df.head(4)

,AttachTo,_IdType,TargetId,ItemId,OrganizationId,ProviderId,MemberId,Attachment,Attachment.1
0,1..1,1..1,1..1,1..*,1..1,1..1,0..1,1..*,1..*
1,code,code,Identifier,string,Identifier,Identifier,Identifier,resource,resource
2,claim,FILL,http://example.org/cdex/claim-ids|11234657,http://example.org/cdex/item-ids|1,http://hl7.org/fhir/sid/us-npi|1407071236,http://hl7.org/fhir/sid/us-npi|9941339108,http://example.org/cdex/member-ids|234567,fhir_templates/docref1.yml,fhir_templates/docref2.yml
3,claim,FILL,http://example.org/cdex/claim-ids|11234657,http://example.org/cdex/item-ids|1,http://hl7.org/fhir/sid/us-npi|1407071236,http://hl7.org/fhir/sid/us-npi|9941339108,http://example.org/cdex/member-ids|234567,fhir_templates/bundle1.yml,nan


In [5]:
from fhir.resources import construct_fhir_element
from fhir.resources.fhirabstractmodel import ValidationError
from pathlib import Path
from fhir.resources.documentreference import DocumentReference
from fhir.resources.bundle import Bundle
from utils import validate
from yaml import load as y_load, dump as y_dump
from json import dumps, loads

def get_type(parameter, attachment_type):
        
    type_coding = construct_fhir_element(
    'Coding',dict(
        system = 'http://terminology.hl7.org/CodeSystem/v2-0203',
        code = attachment_type,
        display = 'Filler Identifier' if attachment_type == "FILL" else 'Placer Identifier',
        )
    )
    
    if parameter in ['TargetId']:
           r_cc = construct_fhir_element(
            'CodeableConcept',dict(
            coding = [type_coding],
            )
        )
    else:
        return
    return r_cc


def get_dr(path):
    dr = DocumentReference.parse_file(path)
    print(help(dr))
    timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
    tzinfo = timezone(timedelta(hours=timezone_offset))
    f_now = datetime.now(tzinfo)
    print(f_now)
    
def get_resource(url): # get direct from yaml file
  try: 
    r = DocumentReference.parse_file(url)
    #print(r)
  except ValidationError as e:
    r = Bundle.parse_file(url)
    #print(e)
  except FileNotFoundError as e:
    print(e)
    return
  return r  

In [6]:
r_obj = construct_fhir_element('Parameters', {})
r_obj.parameter = []
print('-'*80)
for i_name,i_value in df.iteritems():
    print(i_name,i_value[0],i_value[1], df.AttachTo[my_row],)
    if i_name.startswith('_') or i_value[my_row]=="nan": 
        continue
    elif i_value[1] == 'resource':
        param = construct_fhir_element(
            'ParametersParameter', {
                'name': i_name.split(".")[0],  # repeats get numbers appended by pandas
                'resource' :  get_resource(i_value[my_row])  # get direct from yaml file
            }
        )
    elif i_value[1] == 'Identifier':         
        param = construct_fhir_element(
            'ParametersParameter', {
                'name': i_name,
                f'value{i_value[1]}':construct_fhir_element(i_value[1], dict(
                        system = i_value[my_row].split('|')[0] if '|' in i_value[my_row] else None,
                        value = i_value[my_row].split('|')[1] if '|' in i_value[my_row] else i_value[2],
                        type = get_type(i_name,df._IdType[my_row])
                    )
                )         
            }
        )
    else:
            param = construct_fhir_element(
                'ParametersParameter', {
                    'name':i_name,
                    f'value{i_value[1].capitalize()}': i_value[my_row]    
                }
            )

#     print(param.yaml())
    r_obj.parameter.append(param)
print('-'*80)

# validate(r_obj) # no endpoint to validate!
# print('-'*80)
# print(r_obj.json(indent=2))
# instead of save directly as yaml need to round trip to save datetime properly for fhir

r_json = r_obj.json()
r_dict = loads(r_json)
r_yaml = y_dump(r_dict, sort_keys=False)

     
out_path = '/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/includes-yaml/cdex-unsolicited-attachment-example2-body.yml'
path = Path() / out_path
print(f'writing to {path}')
path.write_text(r_yaml)

--------------------------------------------------------------------------------
AttachTo 1..1 code claim
_IdType 1..1 code claim
TargetId 1..1 Identifier claim
ItemId 1..* string claim
OrganizationId 1..1 Identifier claim
ProviderId 1..1 Identifier claim
MemberId 0..1 Identifier claim
Attachment 1..* resource claim
Attachment.1 1..* resource claim
--------------------------------------------------------------------------------
writing to /Users/ehaas/Documents/FHIR/Davinci-CDEX/input/includes-yaml/cdex-unsolicited-attachment-example2-body.yml


15700